# Data Preprocessing

This notebook goal is to preprocess the data for the DonorsChoose.org Kaggle challenge. From the data exploration (EDA.ipynb) it was possible to understand what are the columns that need to be preprocessed and how they should be handled for the data to be eligible has an input to a machine learning model.

I will do the preprocessing of the data dataset by dataset, in the end I will merge all the resultant datasets.

In [81]:
# import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff
from wordcloud import WordCloud
from collections import Counter
import operator
import warnings
warnings.filterwarnings('ignore')
from datetime import datetime
import time
from sklearn.preprocessing import LabelEncoder
from geopy.geocoders import Nominatim
from collections import Counter

from keras.models import Model, load_model
from keras import regularizers, losses

In [82]:
# to import from parallel directory
import os,sys,inspect
current_dir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parent_dir = os.path.dirname(current_dir)
sys.path.insert(0, parent_dir)

from src.aux_functions import *

## Outcomes.csv

In this dataset we will only use the column is_exciting. Since this dataset has only information from the training set, all the other parameters are useless for the model.

Regarding the information on the EDA.ipynb, I will only use the data starting in 01-2010 until 11-2013.

In [83]:
# Load dataset outcomes 
outcomes = pd.read_csv('outcomes.csv')

# Load dataset project to extract date when the project was posted
projects = pd.read_csv('projects.csv')

In [84]:
# Extract only the project after 2009

merged = pd.merge(projects[['projectid','date_posted']], outcomes[['projectid','is_exciting']], on='projectid')
merged['year'] = pd.DatetimeIndex(merged['date_posted']).year
after2010 = merged[merged['year'] > 2009]


In [85]:
after2010.head(n=1)

,projectid,date_posted,is_exciting,year
0,62526d85d2a1818432d03d600969e99c,2013-12-31,f,2013


In [86]:
train_y = after2010[['projectid', 'is_exciting', 'year']]

In [87]:
train_y = train_y.replace({'is_exciting':{'t':1, 'f':0}})


For now I will save the year because if I intend to use the data as validation, the validation year needs to be posterio than the training data.

In [88]:
# Save the data in file train_y.csv
train_y.to_pickle('train_y.csv')

In [89]:
# Save the projectid of the data that will be used as training data
projectidTrain = after2010[['projectid']]

## Projects.csv



In [90]:
# load dataset
projectsOld = pd.read_csv('projects.csv')
projectsOld.head(n=1)


,projectid,teacher_acctid,schoolid,school_ncesid,school_latitude,school_longitude,school_city,school_state,school_zip,school_metro,...,resource_type,poverty_level,grade_level,fulfillment_labor_materials,total_price_excluding_optional_support,total_price_including_optional_support,students_reached,eligible_double_your_impact_match,eligible_almost_home_match,date_posted
0,316ed8fb3b81402ff6ac8f721bb31192,42d43fa6f37314365d08692e08680973,c0e6ce89b244764085691a1b8e28cb81,6.362701e+10,36.57634,-119.608713,Selma,CA,93662.0,NaN,...,Books,highest poverty,Grades 6-8,30.0,555.81,653.89,32.0,f,f,2014-05-12


In [92]:
merged = projects[['projectid','date_posted']]
merged['year'] = pd.DatetimeIndex(merged['date_posted']).year
after2010 = merged[merged['year'] > 2009]
after2010.shape


(484371, 3)

In [93]:
projects = pd.merge(projectsOld, after2010, on='projectid')
projects.shape

(484371, 37)

In [94]:
# columns with nan values, after excluding data before 2010
projects.columns[projects.isna().sum() > 0]

Index(['school_ncesid', 'school_zip', 'school_metro', 'school_district',
       'teacher_prefix', 'primary_focus_subject', 'primary_focus_area',
       'secondary_focus_subject', 'secondary_focus_area', 'resource_type',
       'grade_level', 'students_reached'],
      dtype='object')

In [95]:
# Encoding school_state
# creating instance of labelencoder
labelencoder = LabelEncoder()

# Assigning numerical values and storing in another column
projects['school_state'] = labelencoder.fit_transform(projects['school_state'])


In [96]:
# Encoding 
# creating instance of labelencoder
labelencoder = LabelEncoder()

# Assigning numerical values and storing in another column
projects['teacher_acctid'] = labelencoder.fit_transform(projects['teacher_acctid'])


In [97]:
# Encoding 
# creating instance of labelencoder
labelencoder = LabelEncoder()

# Assigning numerical values and storing in another column
projects['schoolid'] = labelencoder.fit_transform(projects['schoolid'])



In [98]:
# Encoding 
# creating instance of labelencoder
labelencoder = LabelEncoder()

# Assigning numerical values and storing in another column
projects['school_ncesid'] = labelencoder.fit_transform(projects['school_ncesid'])




In [99]:
# Encoding school_city
# creating instance of labelencoder
labelencoder = LabelEncoder()

# Assigning numerical values and storing in another column
projects['school_city'] = labelencoder.fit_transform(projects['school_city'])

In [100]:
# Replace NaN values in column school_zip
locator = Nominatim(user_agent='myGeocoder')

nanSchool_zip = projects[projects['school_zip'].isnull()].index

for i in nanSchool_zip:
    # Use school_latitude and school_longitude columns
    projects['school_zip'][i] = locator.reverse(str(projects['school_latitude'][i]) + \
                                                               ' ,' + str(projects['school_longitude'][i])).raw['address']['postcode']


In [101]:
# Replace NaN values in column school_metro
nanSchool_metro = projects[projects['school_metro'].isnull()].index
c = 0

for i in nanSchool_metro:

    new_value = projects[projects['school_city'] == projects['school_city'][i]]['school_metro'].value_counts()
    
    if len(new_value) <= 0:
        projects['school_metro'][i] = 'other' # corresponding to 'Other'
        continue 
        
    c += 1
    projects['school_metro'][i] = new_value.index[0]

projects[projects['school_metro'].isnull()].index


labelencoder = LabelEncoder()
projects['school_metro'] = labelencoder.fit_transform(projects['school_metro'])




In [102]:
# Remove columns school_district and school_county since they are redundant
del projects['school_district']
del projects['school_county']


In [103]:
# Convert object type column to boolean

projects = projects.replace({'school_charter':{'t':1, 'f':0},
                            'school_magnet':{'t':1, 'f':0},
                            'school_year_round':{'t':1, 'f':0},
                            'school_nlns':{'t':1, 'f':0},
                            'school_kipp':{'t':1, 'f':0},
                            'school_charter_ready_promise':{'t':1, 'f':0},
                            'teacher_teach_for_america':{'t':1, 'f':0},
                            'teacher_ny_teaching_fellow':{'t':1, 'f':0},
                            'eligible_double_your_impact_match':{'t':1, 'f':0},
                            'eligible_almost_home_match':{'t':1, 'f':0}})


In [107]:
# Encoding teacher_prefix
projects['teacher_prefix'] = projects['teacher_prefix'].replace(np.nan, 'Other', regex=True)

labelencoder = LabelEncoder()
projects['teacher_prefix'] = labelencoder.fit_transform(projects['teacher_prefix'])



In [108]:
# Fill NaN of secondary_focus_subject with primary_focus_subject, encode and replace rest of NaN

projects['secondary_focus_subject'].fillna(projects['primary_focus_subject'], inplace=True)
projects['secondary_focus_subject'] = projects['secondary_focus_subject'].replace(np.nan, 'Other', regex=True)

labelencoder = LabelEncoder()
projects['secondary_focus_subject'] = labelencoder.fit_transform(projects['secondary_focus_subject'])



# Fill NaN of secondary_focus_subject with primary_focus_area

projects['secondary_focus_area'].fillna(projects['primary_focus_area'], inplace=True)
projects['secondary_focus_area'] = projects['secondary_focus_area'].replace(np.nan, 'Other', regex=True)

labelencoder = LabelEncoder()
projects['secondary_focus_area'] = labelencoder.fit_transform(projects['secondary_focus_area'])



In [109]:
# Encode primary_focus_subject
labelencoder = LabelEncoder()

projects['primary_focus_subject'] = projects['primary_focus_subject'].replace(np.nan, 'Other', regex=True)
projects['primary_focus_subject'] = labelencoder.fit_transform(projects['primary_focus_subject'])

# Encode primary_focus_area
labelencoder = LabelEncoder()

projects['primary_focus_area'] = projects['primary_focus_area'].replace(np.nan, 'Other', regex=True)
projects['primary_focus_area'] = labelencoder.fit_transform(projects['primary_focus_area'])

# Encode resource_type
labelencoder = LabelEncoder()
projects['resource_type'] = projects['resource_type'].replace(np.nan, 'NaN', regex=True)
projects['resource_type'] = labelencoder.fit_transform(projects['resource_type'])



In [110]:
# Encode poverty_level
labelencoder = LabelEncoder()
projects['poverty_level'] = labelencoder.fit_transform(projects['poverty_level'])



In [111]:
# Encode resource_type
labelencoder = LabelEncoder()
projects['grade_level'] = projects['grade_level'].replace(np.nan, 'NaN', regex=True)
projects['grade_level'] = labelencoder.fit_transform(projects['grade_level'])


In [112]:
projects['fulfillment_labor_materials'].value_counts()

30.0    241102
35.0    221145
9.0      22124
Name: fulfillment_labor_materials, dtype: int64

Regarding the Students reached, lets se how many rows have that column as NaN.

In [113]:
projects['students_reached'].isna().sum()

146

Since there are only 144 rows with no information about the students reached, and this column is revealing useful information, has we have seen on the EDA, I fill the nan values with the mode.

In [114]:
projects['students_reached'].fillna(projects['students_reached'].mode()[0], inplace=True)


In [121]:
# Convert date column to columns Year, Month and Day
projects['year_posted'] = pd.DatetimeIndex(projects['date_posted_x']).year
projects['month_posted'] = pd.DatetimeIndex(projects['date_posted_x']).month
projects['day_posted'] = pd.DatetimeIndex(projects['date_posted_x']).day



In [122]:
# Delete date_posted column
del projects['date_posted_x']
del projects['date_posted_y']


In [123]:
projects.dtypes

projectid                                  object
teacher_acctid                              int64
schoolid                                    int64
school_ncesid                               int64
school_latitude                           float64
school_longitude                          float64
school_city                                 int64
school_state                                int64
school_zip                                float64
school_metro                                int64
school_charter                              int64
school_magnet                               int64
school_year_round                           int64
school_nlns                                 int64
school_kipp                                 int64
school_charter_ready_promise                int64
teacher_prefix                              int64
teacher_teach_for_america                   int64
teacher_ny_teaching_fellow                  int64
primary_focus_subject                       int64


In [124]:
# Save dataset
projects.to_pickle('clean_projects.csv')

## Resources.csv

In [125]:
# load dataset
resourcesOld = pd.read_csv('resources.csv')
projectidTrain = pd.read_pickle('./train_y.csv')[['projectid']]
resourcesOld.head(n=1)


,resourceid,projectid,vendorid,vendor_name,project_resource_type,item_name,item_number,item_unit_price,item_quantity
0,8a1c1c45bc30d065061912fd9114fcf3,ffffc4f85b60efc5b52347df489d0238,430.0,Woodwind and Brasswind,Technology,iPod nano 4th Gen 8GB (Black),249995.001,149.0,4.0


In [126]:
resourcesOld.shape

(3667217, 9)

In [128]:
merged = projectsOld[['projectid','date_posted']]
merged['year'] = pd.DatetimeIndex(merged['date_posted']).year
after2010 = merged[merged['year'] > 2009]
after2010.shape

(484371, 3)

In [129]:
resources = pd.merge(resourcesOld, after2010['projectid'], on='projectid')
resources.shape

(2749806, 9)

In [130]:
resources.head(n=1)

,resourceid,projectid,vendorid,vendor_name,project_resource_type,item_name,item_number,item_unit_price,item_quantity
0,015d2c4935c50427964a12dc3f584091,ffffac55ee02a49d1abc87ba6fc61135,82.0,Best Buy for Business,Technology,Sony bloggie MHS-FS1 - camcorder - internal fl...,BB11216668,148.0,1.0


In [131]:
# Get the amount spent by each project

# multiply column item_unit_price by item_quantity -> item_total_quantity
resources['item_total_quantity'] = resources['item_unit_price'] * resources['item_quantity']

# group resources[['projectid', item_total_quantity]] by projectid and sum values of item_total_quantity
groupedResources =resources.groupby(['projectid']).sum()

In [132]:
# Get main vendor for each projectid

# Count num of vendor ocurrences for each projectid, get the max count 
resources['vendor_name'] = resources['vendor_name'].replace(np.nan, '', regex=True)

s = resources.groupby(['projectid','vendor_name']).size()
majorVendor = s.loc[s.groupby(level=0).idxmax()].reset_index().drop(0,axis=1)


In [133]:
# Delete columns item_name, item_number, item_unit_price, item_quantity, vendorid, resourceid, project_resource_type
newResources =  pd.merge(groupedResources[[ 'item_quantity', 'item_total_quantity']], majorVendor, on='projectid')


In [134]:
newResources.head(n=3)

,projectid,item_quantity,item_total_quantity,vendor_name
0,00001ccc0e81598c4bd86bacb94d7acb,55.0,1225.44,Lakeshore Learning Materials
1,00002bff514104264a6b798356fdd893,7.0,399.74,Lakeshore Learning Materials
2,00002d691c05c51a5fdfbb2baef0ba25,122.0,774.80,AKJ Books


In [135]:
# Encode 
labelencoder = LabelEncoder()
newResources['vendor_name'] = labelencoder.fit_transform(newResources['vendor_name'])


In [136]:
newResources.dtypes

projectid               object
item_quantity          float64
item_total_quantity    float64
vendor_name              int64
dtype: object

In [137]:
newResources.shape

(484371, 4)

In [138]:
# ToDo: Save clean dataset 
newResources.to_pickle('clean_resources.csv')

## Essays.csv



In [139]:
# Load dataset essays
essaysOld = pd.read_csv('essays.csv')
essaysOld.head(n=1)

,projectid,teacher_acctid,title,short_description,need_statement,essay
0,ffffc4f85b60efc5b52347df489d0238,c24011b20fc161ed02248e85beb59a90,iMath,It is imperative that teachers bring technolog...,My students need four iPods.,I am a fourth year fifth grade math teacher. T...


In [140]:
essaysOld.shape

(664098, 6)

In [141]:
merged = projectsOld[['projectid','date_posted']]
merged['year'] = pd.DatetimeIndex(merged['date_posted']).year
after2010 = merged[merged['year'] > 2009]
after2010.shape

(484371, 3)

In [142]:
# Remove the projects posted before 2010
essays = pd.merge(essaysOld, after2010, on='projectid')

In [143]:
essays.shape

(484371, 8)

In [144]:
# Replace the NaN by empty strings
essays = essays.replace(np.nan, '', regex=True)

In [145]:
# Load pretrained MLP model 
mlp_model = load_model('/Users/anacosta/Desktop/KaggleComp_DonorsChoose/'+'saved_models/mlp.h5', custom_objects={'custom_loss':custom_loss})

fast = load_fastext()

#### Title column

In this column, the goal is to do a dimensionl sentiment analysis (considering the dimensions Valence and Arousal). The column <b>title</b> will produce the columns <b>title_v</b> and <b>title_a</b>.

In [146]:
# Preprocess the column Title
t0 = time.time() # track time

# tokenize the text
tokenize_text(essays, 'title')

# clean, lemmatize, get embeddings and predict sentiment
predict_sentiment(essays, 'title', mlp_model, fast)

t1 = time.time()

total = t1-t0
total/60

14.123394648234049

In [147]:
# fill NaN Rows with [0,0,0]
essays.loc[essays['title'].isnull(),['title']] = essays.loc[essays['title'].isnull(),'title'].apply(lambda x: list([0,0,0]))


In [148]:
# separate column into 3
aux = pd.DataFrame(essays['title'].to_list(), columns=['title_v','title_a','title_d'])

In [149]:
# Append title_v and title_a to dataset
newEssays = pd.merge(aux[['title_v', 'title_a']], essays, left_index=True, right_index=True)

In [150]:
# Remove Title 
del newEssays['title']
newEssays.head(n=1)

,title_v,title_a,projectid,teacher_acctid,short_description,need_statement,essay,date_posted,year
0,0.795637,0.521858,ffffac55ee02a49d1abc87ba6fc61135,947066d0af47e0566f334566553dd6a6,Can you imagine having to translate everything...,My students need a camcorder.,Can you imagine having to translate everything...,2011-06-11,2011


In [151]:
# Saving all the intermediate steps because it takes a long time to run
newEssays.to_pickle('essaysTitle.csv')

#### Short description Column

Get a column with the number of words of the short description (<b>numb_words_short_description</b>)

In [152]:
newEssays = pd.read_pickle('./essaysTitle.csv')

newEssays = newEssays.replace(np.nan, '', regex=True)

In [153]:
# Preprocess the column short_description

# tokenize the text
tokenize_text(newEssays, 'short_description')


0         [can, you, imagine, having, to, translate, eve...
1         [it, takes, a, special, person, to, donate, to...
2         [my, kindergarten, students, come, from, a, va...
3         [have, you, ever, had, to, crowd, around, an, ...
4         [the, rewards, of, playing, a, musical, instru...
                                ...                        
484366    [kindergarten, is, an, exciting, time, for, le...
484367    [my, students, have, very, limited, exposure, ...
484368    [my, students, need, high, quality, books, suc...
484369    [my, students, need, more, incentives, to, mak...
484370    [my, students, would, love, the, opportunity, ...
Name: short_description, Length: 484371, dtype: object

In [154]:
# Get column numb_words_short_description
# count the number of words
newEssays['numb_words_short_description'] = newEssays['short_description'].apply(lambda x: len(x))


In [155]:
clean_stopwords(newEssays, 'short_description')
lematize(newEssays, 'short_description')

0         [imagine, translate, everything, read, sign, l...
1         [take, special, person, donate, group, child, ...
2         [kindergarten, student, come, variety, backgro...
3         [ever, crowd, around, ipad, screen, access, in...
4         [reward, playing, musical, instrument, great, ...
                                ...                        
484366    [kindergarten, exciting, time, learning, espec...
484367    [student, limited, exposure, technology, techn...
484368    [student, need, high, quality, book, worth, ch...
484369    [student, need, incentive, make, better, faste...
484370    [student, would, love, opportunity, create, ma...
Name: short_description, Length: 484371, dtype: object

In [156]:
# Count number of words
counter = Counter()
_ = newEssays['short_description'].apply(lambda x: counter.update(x))


This part was commented because it takes too much time to run.

In [157]:
'''
# Get column uniq_words_short_description

# compute unique words

uniqueWords = [key  for (key, value) in counter.items() if value == 1]

# count unique words each row
notUniq = newEssays['short_description'].apply(lambda x: pd.Series(x).isin(uniqueWords).value_counts())
#newEssays['short_description'].isin(uniqueWords).value_counts()[True]

newEssays['uniq_words_short_description'] = newEssays['numb_words_short_description'] - notUniq.squeeze()
'''

"\n# Get column uniq_words_short_description\n\n# compute unique words\n\nuniqueWords = [key  for (key, value) in counter.items() if value == 1]\n\n# count unique words each row\nnotUniq = newEssays['short_description'].apply(lambda x: pd.Series(x).isin(uniqueWords).value_counts())\n#newEssays['short_description'].isin(uniqueWords).value_counts()[True]\n\nnewEssays['uniq_words_short_description'] = newEssays['numb_words_short_description'] - notUniq.squeeze()\n"

In [158]:
# Get column most_common_short_description

most_common50 = counter.most_common(100) 


In [159]:
most_common_short_description = newEssays['short_description'].apply(lambda x: pd.Series(x).isin(most_common50).value_counts())



In [160]:
newEssays['most_common_short_description'] = newEssays['numb_words_short_description'] - most_common_short_description.squeeze()


In [161]:
# Saving all the intermediate steps because it takes a long time to run
newEssays.to_pickle('essaysShortDescript.csv')


#### Essay Column

Since treating text is too expensive, from this column we will only extract the number of words that each essay has and then will delete the Essay column from this dataset.

In [162]:
newEssays = pd.read_pickle('./essaysShortDescript.csv')

newEssays = newEssays.replace(np.nan, '', regex=True)


In [163]:
# Preprocess the column essays

# tokenize the text
tokenize_text(newEssays, 'essay')

0         [can, you, imagine, having, to, translate, eve...
1         [hi, i, teach, a, wonderful, group, of, 4, 5, ...
2         [my, kindergarten, students, come, from, a, va...
3         [have, you, ever, had, to, crowd, around, an, ...
4         [the, rewards, of, playing, a, musical, instru...
                                ...                        
484366    [kindergarten, is, an, exciting, time, for, le...
484367    [my, students, have, very, limited, exposure, ...
484368    [our, students, need, the, challenge, to, read...
484369    [my, students, need, more, incentives, to, mak...
484370    [my, students, would, love, the, opportunity, ...
Name: essay, Length: 484371, dtype: object

In [164]:
# Get column numb_words_essay
# count the number of words
newEssays['numb_words_essay'] = newEssays['essay'].apply(lambda x: len(x))


In [165]:
newEssays.head(n=1)

,title_v,title_a,projectid,teacher_acctid,short_description,need_statement,essay,date_posted,year,numb_words_short_description,most_common_short_description,numb_words_essay
0,0.795637,0.521858,ffffac55ee02a49d1abc87ba6fc61135,947066d0af47e0566f334566553dd6a6,"[imagine, translate, everything, read, sign, l...",My students need a camcorder.,"[can, you, imagine, having, to, translate, eve...",2011-06-11,2011,36,20,199


In [ ]:
# Remove columns 
del newEssays['short_description']
del newEssays['need_statement']
del newEssays['essay']
del newEssays['teacher_acctid']



In [174]:
del newEssays['date_posted']

In [183]:
newEssays['most_common_short_description'] = newEssays['most_common_short_description'].replace('', 0, regex=True)


In [184]:
newEssays.dtypes

title_v                          float64
title_a                          float64
projectid                         object
year                               int64
numb_words_short_description       int64
most_common_short_description    float64
numb_words_essay                   int64
dtype: object

In [185]:
# Saving clean essays
newEssays.to_pickle('clean_essays.csv')


## Merge all datasets into Train X

In [186]:
clean_Essays = pd.read_pickle('./clean_essays.csv')

clean_Projects = pd.read_pickle('./clean_projects.csv')

clean_Resources = pd.read_pickle('./clean_resources.csv')


In [187]:
clean_Essays.shape


(484371, 7)

In [188]:
clean_Projects.shape


(484371, 36)

In [189]:
clean_Resources.shape

(484371, 4)

In [190]:
mergedX = pd.merge(clean_Essays, clean_Projects, on='projectid')


In [191]:
mergedX = pd.merge(mergedX, clean_Resources, on='projectid')


In [192]:
mergedX.shape

(484371, 45)

In [201]:
mergedX.columns

Index(['title_v', 'title_a', 'projectid', 'year_x',
       'numb_words_short_description', 'most_common_short_description',
       'numb_words_essay', 'teacher_acctid', 'schoolid', 'school_ncesid',
       'school_latitude', 'school_longitude', 'school_city', 'school_state',
       'school_zip', 'school_metro', 'school_charter', 'school_magnet',
       'school_year_round', 'school_nlns', 'school_kipp',
       'school_charter_ready_promise', 'teacher_prefix',
       'teacher_teach_for_america', 'teacher_ny_teaching_fellow',
       'primary_focus_subject', 'primary_focus_area',
       'secondary_focus_subject', 'secondary_focus_area', 'resource_type',
       'poverty_level', 'grade_level', 'fulfillment_labor_materials',
       'total_price_excluding_optional_support',
       'total_price_including_optional_support', 'students_reached',
       'eligible_double_your_impact_match', 'eligible_almost_home_match',
       'year_y', 'year_posted', 'month_posted', 'day_posted', 'item_quantity',
  

In [202]:
mergedX['year'] = mergedX['year_x']

del mergedX['year_x']
del mergedX['year_y']


In [207]:
train_mergedX = mergedX[mergedX['year'] < 2014]

# Saving train dataset
train_mergedX.to_pickle('train_x.csv')


In [209]:
test_mergedX = mergedX[mergedX['year'] < 2014]

# Saving train dataset
test_mergedX.to_pickle('test_x.csv')
